# Lab 7 —  SDM optimization

### Tomasz Ogiołda

In [8]:
from types import SimpleNamespace
import numpy as np
import scipy
import numba

In [9]:
PARAMS_PHYS = SimpleNamespace(
    n0=2**23,
    dv_m3 = 1e6,
    x0_kg=1.192e-13,
    b_per_s = 1500,
)
PARAMS_PHYS.dist = scipy.stats.expon(loc=0, scale=PARAMS_PHYS.x0_kg)
PARAMS_PHYS.norm = PARAMS_PHYS.n0 * PARAMS_PHYS.dv_m3

PARAMS_COMP = SimpleNamespace(
    n_part = 2**11,
    t_max_s = 3600,
    n_step = 1800,
)
PARAMS_COMP.dt_s = PARAMS_COMP.t_max_s / PARAMS_COMP.n_step

PARAMS_BINS = SimpleNamespace(
    min_x = -12,
    max_x = -5,
    count = 70
)

RNG = np.random.default_rng(seed=44)

In [10]:
def sample(*, params_comp, params_phys, rng):
    """ randomly samples a particle population using constant-multiplicity,
    uniform-mass and uniform-log-mass schemes and returns a dictionary
    of three simulation state, each composed of 'mass' and 'mult' arrays """
    u01 = rng.uniform(0, 1, size=params_comp.n_part)
    uniform_sampling_range = [params_phys.dist.ppf(q) for q in (.001, .999)]
    x_uniform_linx = (
        uniform_sampling_range[0]
        + u01 * (uniform_sampling_range[1] - uniform_sampling_range[0])
    )
    x_uniform_logx = np.exp(
        np.log(uniform_sampling_range[0])
        + u01 * (np.log(uniform_sampling_range[1]) - np.log(uniform_sampling_range[0]))
    )
    return {
        k: {
            'mass': v['x'],
            'mult': np.round(v['y'] * params_phys.norm).astype(int),
        }
        for k,v in
        {
            'sampling: uniform random in x': {
                'x': x_uniform_linx,
                'y': params_phys.dist.pdf(x_uniform_linx) \
                  * (uniform_sampling_range[1] - uniform_sampling_range[0]) \
                  / params_comp.n_part,
            },
            'sampling: uniform random in ln(x)': {
                'x': x_uniform_logx,
                'y': params_phys.dist.pdf(x_uniform_logx) \
                  * (np.log(uniform_sampling_range[1]) - np.log(uniform_sampling_range[0])) \
                  / (params_comp.n_part / x_uniform_logx),
            },
            'sampling: constant multiplicity': {
                'x': params_phys.dist.ppf(u01),
                'y': np.full(shape=params_comp.n_part, fill_value=1 / params_comp.n_part),
            }
        }.items()
    }

PARTICLES = sample(params_phys=PARAMS_PHYS, params_comp=PARAMS_COMP, rng=RNG)

In [11]:
def x_of_mass(mass):
    """ defines the plot x coordinate as a funciton of particle mass """
    return np.log(mass) / 3

def mass_of_x(coord):
    """ computes mass back from the plot x coordinate """
    return np.exp(3 * coord)

def kernel(mass_1, mass_2, coeff):
    """ additive coagulation kernel """
    return coeff * (mass_1 + mass_2)

def analytic_solution(mass_kg, time_s, params_phys):
    """ Golovin's analytic solution to Smoluchowski coagulation equation 
    for additive kernel and exponential initial condition """
    tau = 1 - np.exp(-params_phys.n0 * params_phys.b_per_s * params_phys.x0_kg * time_s)
    sqrt_tau = np.sqrt(tau)
    return (
        (1 - tau) / (mass_kg * sqrt_tau)
        * scipy.special.ive(1, 2 * mass_kg / params_phys.x0_kg * sqrt_tau)  # pylint: disable=no-member
        * np.exp(-(1 + tau - 2 * sqrt_tau) * mass_kg / params_phys.x0_kg)
    )

In [12]:
def sdm(*, pairs, u01, mult, mass, b_per_s, dt, dv):
    """ performs Monte-Carlo coagulation using the SDM algorithm """
    for alpha, (j, k) in enumerate(pairs):
        p_jk = kernel(mass[j], mass[k], coeff=b_per_s) * dt / dv
        p_s_jk = max(mult[j], mult[k]) * p_jk
        n = len(mult)
        p_alpha = p_s_jk * n * (n - 1) / (2 * len(pairs))
        
        # assert p_alpha < 1 and p_alpha > 0
        # TODO: Gamma factor fixing assertion in GH repo from the labs
        
        
        if u01[alpha] < p_alpha:
            if mult[j] < mult[k]:
                j, k = k, j
                
            if mult[j] != mult[k]:
                mult[j] -= mult[k]
                mass[k] += mass[j]
            else:
                mult[k] = mult[j] - mult[j] // 2
                mult[j] = mult[j] // 2
                mass[k] += mass[j]
                mass[j] = mass[k]

In [13]:
@numba.cuda.jit(parallel=True)
def kernel(mass_1, mass_2, coeff):
    """ additive coagulation kernel """
    return coeff * (mass_1 + mass_2)

@numba.cuda.jit(parallel=True)
def sdm_optimized(*, pairs, u01, mult, mass, b_per_s, dt, dv):
    """ performs Monte-Carlo coagulation using the SDM algorithm """
    for alpha, (j, k) in enumerate(pairs):
        p_jk = kernel(mass[j], mass[k], coeff=b_per_s) * dt / dv
        p_s_jk = max(mult[j], mult[k]) * p_jk
        n = len(mult)
        p_alpha = p_s_jk * n * (n - 1) / (2 * len(pairs))
        
        # assert p_alpha < 1 and p_alpha > 0
        # TODO: Gamma factor fixing assertion in GH repo from the labs
        
        
        if u01[alpha] < p_alpha:
            if mult[j] < mult[k]:
                j, k = k, j
                
            if mult[j] != mult[k]:
                mult[j] -= mult[k]
                mass[k] += mass[j]
                
            else:
                mult[k] = mult[j] - mult[j] // 2
                mult[j] = mult[j] // 2
                mass[k] += mass[j]
                mass[j] = mass[k]

AttributeError: module 'numba' has no attribute 'cuda'

In [28]:
def simulate(*, params_phys, params_comp, particles, rng, sdm=sdm):
    """ does simulation for all sampling variants (each variant using the same shuffled numbers) """
    n_pairs = params_comp.n_part // 2
    for _ in range(params_comp.n_step):
        non_overlapping_pairs = rng.permutation(params_comp.n_part)[: 2 * n_pairs].reshape(-1, 2)
        u01 = rng.uniform(0, 1, n_pairs)
        for part in particles.values():
            sdm(
                pairs=non_overlapping_pairs,
                u01=u01,
                b_per_s=params_phys.b_per_s,
                dt=params_comp.dt_s,
                dv=params_phys.dv_m3,
                **part
            )

In [35]:
simulate(
    particles=PARTICLES, 
    params_phys=PARAMS_PHYS, 
    params_comp=PARAMS_COMP, 
    rng=RNG,
    sdm=sdm_optimized
)

In [40]:
# Benchmarks
import time

def measure_perf(sim_fn):
    start_time = time.perf_counter()
    sim_fn()
    end_time = time.perf_counter()
    duration = end_time - start_time
    
    return (start_time, end_time, duration)
    
    
raw_sdm = measure_perf(lambda : simulate(
    particles=PARTICLES.copy(), 
    params_phys=PARAMS_PHYS, 
    params_comp=PARAMS_COMP, 
    rng=RNG,
    sdm=sdm
))
optimized_sdm = measure_perf(lambda : simulate(
    particles=PARTICLES.copy(), 
    params_phys=PARAMS_PHYS, 
    params_comp=PARAMS_COMP, 
    rng=RNG,
    sdm=sdm_optimized
))
print(f'Raw: {raw_sdm}')
print(f'Optimized: {optimized_sdm}')

sdm_optimized.parallel_diagnostics(level=4)

Raw: (2027481.661168917, 2027504.91451775, 23.253348832949996)
Optimized: (2027504.914711333, 2027505.037753042, 0.12304170895367861)
 
 Parallel Accelerator Optimizing:  Function sdm_optimized, 
/var/folders/fz/mfw06hh53blcf3hcdj6s8n3r0000gn/T/ipykernel_9696/2113441850.py 
(8)  
No source available
--------------------------------- Fusing loops ---------------------------------
Attempting fusion of parallel loops (combines loops with similar properties)...
----------------------------- Before Optimisation ------------------------------
--------------------------------------------------------------------------------
------------------------------ After Optimisation ------------------------------
Parallel structure is already optimal.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
 
---------------------------Loop invariant code motion---------------------------
Allocation 